In [2]:
from scipy.stats import kstest
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import chardet
import pycountry
import csv
from pathlib import Path
from all_functions import refactorGiniWorldBankDf

In [20]:
revenuMondial = pd.read_csv("../data/data-projet7.csv", parse_dates=True)
revenuMondial

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297
...,...,...,...,...,...,...
11594,COD,2008,96,100,"810,6233","303,19305"
11595,COD,2008,97,100,"911,7834","303,19305"
11596,COD,2008,98,100,"1057,8074","303,19305"
11597,COD,2008,99,100,"1286,6029","303,19305"


In [ ]:
# Ci-dessus, les revenues mondiale pour une période donnée

In [ ]:
# Premère étape. je veux connaitre les noms de ces pays 

In [28]:
isoPath = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
codes = pd.readr_csv(isoPath)
countryCodes = codes[['name','alpha-3', 'region', 'sub-region']]
countryCodes = countryCodes.rename(columns={"alpha-3":"iso"})
countryCodes

,name,iso,region,sub-region
0,Afghanistan,AFG,Asia,Southern Asia
1,Åland Islands,ALA,Europe,Northern Europe
2,Albania,ALB,Europe,Southern Europe
3,Algeria,DZA,Africa,Northern Africa
4,American Samoa,ASM,Oceania,Polynesia
...,...,...,...,...
244,Wallis and Futuna,WLF,Oceania,Polynesia
245,Western Sahara,ESH,Africa,Northern Africa
246,Yemen,YEM,Asia,Western Asia
247,Zambia,ZMB,Africa,Sub-Saharan Africa


In [39]:
merged_df = pd.merge(revenuMondial, countryCodes, left_on='country', right_on='iso', how='left')
incomes = merged_df[['name','country', 'year_survey', 'quantile','nb_quantiles','income','gdpppp']]
incomes = incomes.rename(columns={"name":"nation"})
incomes = incomes.rename(columns={"year_survey":"year"})
incomesStatus = incomes.copy()
incomes

,nation,country,year,quantile,nb_quantiles,income,gdpppp
0,Albania,ALB,2008,1,100,"728,89795",7297
1,Albania,ALB,2008,2,100,"916,66235",7297
2,Albania,ALB,2008,3,100,"1010,916",7297
3,Albania,ALB,2008,4,100,"1086,9078",7297
4,Albania,ALB,2008,5,100,"1132,6997",7297
...,...,...,...,...,...,...,...
11594,"Congo, Democratic Republic of the",COD,2008,96,100,"810,6233","303,19305"
11595,"Congo, Democratic Republic of the",COD,2008,97,100,"911,7834","303,19305"
11596,"Congo, Democratic Republic of the",COD,2008,98,100,"1057,8074","303,19305"
11597,"Congo, Democratic Republic of the",COD,2008,99,100,"1286,6029","303,19305"


In [ ]:
# Information dataFrame

In [45]:
incomesStatus["year"] = incomesStatus["year"].astype(int)

# Résumé des dates minimales et maximales
date_min = incomesStatus['year'].min()
date_max = incomesStatus['year'].max()

# Résumé des revenus minimales et maximales
incomeMin = incomesStatus['income'].min()
incomeMax = incomesStatus['income'].max()
rowMinIncome = incomesStatus.loc[incomesStatus['income'] == incomeMin]
rowMaxIncome = incomesStatus.loc[incomesStatus['income'] == incomeMax]

# Récupérer le pays associé aux revenus minimaux et maximaux
countryMinIncome = rowMinIncome['nation'].values[0]  # Prendre la première valeur car il pourrait y avoir plusieurs lignes
countryMaxIncome = rowMaxIncome['nation'].values[0]

# Affichage des résultats
print(f"Pays avec le revenu minimal : {countryMinIncome} {incomeMin}")
print(f"Pays avec le revenu maximal : {countryMaxIncome} {incomeMax}")

# Affichage des résultats
print(f"Date minimale : {date_min}")
print(f"Date maximale : {date_max}")


Pays avec le revenu minimal : Central African Republic 100,00194
Pays avec le revenu maximal : Denmark 9998,153
Date minimale : 2004
Date maximale : 2011


In [60]:
num_countries_by_year = incomesStatus.groupby('year')['country'].nunique()
print("Nombre de pays par année:")
print(num_countries_by_year)

Nombre de pays par année:
year
2004     1
2006     5
2007    15
2008    76
2009    12
2010     6
2011     1
Name: country, dtype: int64


In [67]:
years_count_by_country = incomesStatus.groupby('country')['year'].nunique().reset_index(name='count')

# Filtrer les pays ayant au moins trois années
countries_with_at_least_three_years = years_count_by_country[years_count_by_country['count'] >= 1]

# Afficher les résultats
print("Pays avec au moins trois années:")
print(countries_with_at_least_three_years)

Pays avec au moins trois années:
    country  count
0       ALB      1
1       ARG      1
2       ARM      1
3       AUT      1
4       AZE      1
..      ...    ...
111     VEN      1
112     VNM      1
113     XKX      1
114     YEM      1
115     ZAF      1

[116 rows x 2 columns]


In [63]:
incomesStatus['country'].nunique()

116

Traitement des donnée manquante

In [53]:
# Grouper par pays
groupedByCountry = incomesStatus.groupby('country')

# Liste des quantiles attendus (1 à 100)
expectedQuantiles = set(range(1, 101))

# Créer une liste pour stocker les pays avec des quantiles manquants
countriesWithMissingQuantiles = []

# Parcourir chaque groupe (pays) et vérifier les quantiles manquants
for country, group in groupedByCountry:
    actualQuantiles = set(group['quantile'])
    missingQuantiles = expectedQuantiles - actualQuantiles
    
    if missingQuantiles:
        countriesWithMissingQuantiles.append((country, missingQuantiles))

# Afficher les résultats
if countriesWithMissingQuantiles:
    print("Pays avec des quantiles manquants :")
    for country, missingQuantiles in countriesWithMissingQuantiles:
        print(f"{country}: Quantiles manquants {missingQuantiles}")
else:
    print("Aucun pays avec des quantiles manquants.")

Pays avec des quantiles manquants :
LTU: Quantiles manquants {41}


In [56]:
# Vérifier les valeurs manquantes dans le DataFrame
missingValues = incomesStatus.isnull().sum()

columnsWithMissingValues = missingValues[missingValues > 0].index.tolist()
if columnsWithMissingValues:
    print("Colonnes avec des valeurs manquantes :")
    for column in columnsWithMissingValues:
        rows_with_missing_values = incomesStatus[incomesStatus[column].isnull()]
        countries_with_missing_values = rows_with_missing_values['country'].unique()
        
        print(f"{column}: {missingValues[column]} valeur(s) manquante(s) dans les pays suivants:")
        for country in countries_with_missing_values:
            print(f" - {country}")
else:
    print("Aucune valeur manquante dans le DataFrame.")


Colonnes avec des valeurs manquantes :
nation: 100 valeur(s) manquante(s) dans les pays suivants:
 - XKX
gdpppp: 200 valeur(s) manquante(s) dans les pays suivants:
 - XKX
 - PSE


In [3]:
## normes iso countries
iso3 = pd.read_csv("../data/base_countries_iso3.csv", parse_dates=True).reset_index()
iso3 = iso3[['name','alpha-2','alpha-3']]
iso3 = iso3.rename(columns={"alpha-3":"iso"})
iso3


FileNotFoundError: [Errno 2] No such file or directory: '../data/base_countries_iso3.csv'

# Revenue de distribution mondial

In [5]:
incomes = pd.read_csv(data + "worldincome.csv", parse_dates=True)
#baseCountry = baseCountry.rename(columns={'country': 'iso'})
incomes.describe()
incomes

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297
...,...,...,...,...,...,...
11594,COD,2008,96,100,"810,6233","303,19305"
11595,COD,2008,97,100,"911,7834","303,19305"
11596,COD,2008,98,100,"1057,8074","303,19305"
11597,COD,2008,99,100,"1286,6029","303,19305"


In [6]:
incomes["year_survey"].unique() # timeline in df 

array([2008, 2009, 2010, 2007, 2006, 2011, 2004])

In [7]:
representCountry = len(incomes)
totalCountriesInWorld = 195
populationCoveradeByDf = (representCountry / totalCountriesInWorld) * 100
populationCoveradeByDf

5948.205128205128

On resence 195 pays dans le monde le dataset représente 160 pays nous avons 59% qui sont représenter dans le dataset

nous allons clear le dataset voir si il y a des valeurs manquantes etc ..

In [8]:
dfDate = incomes.copy()
dfDate.drop_duplicates(subset='year_survey', keep='first',inplace=True)
dfDate.sort_values(by='year_survey') # la frise chrnologique commence en 2004 et fini en 2011 il manque les donnée de 2005

,country,year_survey,quantile,nb_quantiles,income,gdpppp
9899,SYR,2004,1,100,"143,18066",4512
3600,GHA,2006,1,100,"63,157047","1380,1182"
900,BIH,2007,1,100,"808,3991",7472
0,ALB,2008,1,100,"728,89795",7297
600,BFA,2009,1,100,"138,40517","1085,5139"
700,BGD,2010,1,100,"276,93503",1233
3900,GTM,2011,1,100,"38,463615",4367


In [ ]:
incomes.info()

In [9]:
incomes.replace(',','.', regex=True, inplace=True)
incomes['income'] = incomes['income'].astype(float)
incomes['gdpppp'] = incomes['gdpppp'].astype(float)



In [23]:
incomes[incomes['country']=='ALB'].sort_values(by='income') # j'ai pris le premiers pays du dataset pour connaitre le nombre de quantile 
## nb quantiles toujours = 100

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.0
1,ALB,2008,2,100,916.66235,7297.0
2,ALB,2008,3,100,1010.91600,7297.0
3,ALB,2008,4,100,1086.90780,7297.0
4,ALB,2008,5,100,1132.69970,7297.0
...,...,...,...,...,...,...
95,ALB,2008,96,100,6231.09960,7297.0
96,ALB,2008,97,100,6822.31840,7297.0
97,ALB,2008,98,100,7775.89100,7297.0
98,ALB,2008,99,100,9351.75100,7297.0


In [11]:
numberOfCountry = incomes.groupby('country')
numberOfCountry.count() # nb pays 

,year_survey,quantile,nb_quantiles,income,gdpppp
country,,,,,
ALB,100,100,100,100,100
ARG,100,100,100,100,100
ARM,100,100,100,100,100
AUT,100,100,100,100,100
AZE,100,100,100,100,100
...,...,...,...,...,...
VEN,100,100,100,100,100
VNM,100,100,100,100,100
XKX,100,100,100,100,0


In [12]:
incomes.count() ## si il y a 116 il manque une ligne

country         11599
year_survey     11599
quantile        11599
nb_quantiles    11599
income          11599
gdpppp          11399
dtype: int64

In [13]:
countQuantile = incomes.pivot_table(index='country').sort_values(by='quantile').reset_index()[['country', 'quantile']]
diffQuantile = countQuantile[countQuantile['quantile'] != 50.50000]
diffQuantile ## quantile manquant

,country,quantile
115,LTU,50.59596


In [14]:
ltu = incomes[incomes['country'] =='LTU']
## voir la ligne manquante 
##https://www.appsloveworld.com/numpy/100/21/check-the-missing-integers-from-a-range-in-python
missedQuantile = np.setdiff1d(list(range(1, 101)), list(ltu['quantile']))

missedQuantile

array([41])

In [15]:
if len(missedQuantile) != 0: 
    ## on fait une moyenne de ces deux valeurs 
    getRowsLtu = ltu[ltu['quantile'].isin([missedQuantile[0]-1,missedQuantile[0]+1])]
    getRowsLtu
    ## creation de la row 
    meanRowLtu = getRowsLtu.pivot_table(index='country').reset_index()
    ## insérer la row manquante
    incomes = pd.concat([incomes, meanRowLtu]).reset_index().drop(columns='index')

   

In [16]:
## valeur manquantes gdppppp
showMissGdp = incomes[incomes['gdpppp'].isnull()]
showMissGdp

percentDataMissing = round(len(showMissGdp) / len(incomes) * 100, 1) 
percentDataMissing # les valeurs manquantes gdpppp represente 1.7 % du dataset on décide de les supprimer

showMissGdp


,country,year_survey,quantile,nb_quantiles,income,gdpppp
5800,XKX,2008,1,100,437.89370,NaN
5801,XKX,2008,2,100,508.17133,NaN
5802,XKX,2008,3,100,591.82820,NaN
5803,XKX,2008,4,100,668.00000,NaN
5804,XKX,2008,5,100,730.40220,NaN
...,...,...,...,...,...,...
11294,PSE,2009,96,100,2763.88480,NaN
11295,PSE,2009,97,100,3077.83330,NaN
11296,PSE,2009,98,100,3449.22240,NaN
11297,PSE,2009,99,100,4165.99700,NaN


In [17]:
if showMissGdp['gdpppp'].isnull().sum() > 0:
    incomes.drop(showMissGdp.index, inplace=True)
    

In [18]:
## on enregistre le dataset clear pour la suite de nos exercices

In [19]:
incomes.reset_index()
incomes.to_csv('../data/data-confirm/worldincome_clear.csv', index=False)
incomes


,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.00000
1,ALB,2008,2,100,916.66235,7297.00000
2,ALB,2008,3,100,1010.91600,7297.00000
3,ALB,2008,4,100,1086.90780,7297.00000
4,ALB,2008,5,100,1132.69970,7297.00000
...,...,...,...,...,...,...
11595,COD,2008,97,100,911.78340,303.19305
11596,COD,2008,98,100,1057.80740,303.19305
11597,COD,2008,99,100,1286.60290,303.19305
11598,COD,2008,100,100,2243.12260,303.19305


#  DATA GINI

In [ ]:
# pour lire le dataset on doit le clear en amont car pandas ne lis pas bien le DF

In [213]:
name = "../data/API_SI.POV.GINI_DS2_en_csv_v2_4770433.csv";
file = open(name, "r" , encoding='utf-8-sig')
data = list(csv.reader(file, delimiter=";"))
fx = pd.read_csv(name)
fx.to_csv('../data/gini_first_step.csv', index=False)
file.close()

In [214]:
path = Path('../data/gini_first_step.csv')
if path.is_file() == True :
    gini = pd.read_csv('../data/gini_first_step.csv')

In [215]:
gini.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN,NaN,NaN


In [216]:
# pays manquants 
giniCountry = gini.copy()
giniCountry = giniCountry.rename(columns={'Country Name': 'iso'})
giniCountry = giniCountry['iso'].reset_index()[['iso']]
giniCountry.head()

,iso
0,ABW
1,AFE
2,AFG
3,AFW
4,AGO


In [217]:
missingCountryFromIsoDf = iso3[~iso3['iso'].isin(giniCountry['iso'])]
missingCountryFromIsoDf.head()

,name,alpha-2,iso
1,Åland Islands,AX,ALA
7,Anguilla,AI,AIA
8,Antarctica,AQ,ATA
27,"Bonaire, Sint Eustatius and Saba",BQ,BES
30,Bouvet Island,BV,BVT


In [218]:
pays_absents.count() # 34 pays 
# seul taiwan qui peux être considérer comme un pays les autres sont des regions 

name       34
alpha-2    34
iso        34
dtype: int64

In [221]:
## https://data.worldbank.org/ source des données pour le pays manquant 

taiwanExist = 'TWN' in gini['Country Name'].unique();

if  taiwanExist == False : 

    taiwanGiniValues = {'Country Name':"TWN",'Country Code':"Gini index","Indicator Name":"insert value", "Indicator Code":np.nan,
                        'Country Code':"Gini index",'1960':33.6, '1970':33.6, '1980':32.6 ,
                        '1990':31.3, '2000':28.1, '2010':28.3,
                        '2019':31.2}

    for annee in taiwanGiniValues:
        gini.loc['TWN', annee] = taiwanGiniValues[annee]

    gini = gini.reset_index()
    gini = gini.drop(['index'], axis=1)

gini ## 266 ligne à 267  

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,YEM,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,ZAF,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,ZMB,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,57.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,ZWE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,44.3,NaN,50.3,NaN,NaN,NaN


In [222]:
gini.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN,NaN,NaN


In [231]:
test = refactorGiniWorldBank(gini)
test

NameError: name 'refactorGiniWorldBank' is not defined

In [223]:
def refactorGiniDate(gini): 
    giniData = gini.copy()

    if 'Country Name' in giniData.columns and 'Country Code' in giniData.columns and 'Indicator Name' in giniData.columns and 'Indicator Code' in giniData.columns : 
        getOnlyDate = giniData.drop(columns=['Country Name', 'Country Code','Indicator Name','Indicator Code'])

        firstDateColumnGini = int(getOnlyDate.columns[0])
        lastDataColumnGini = int(getOnlyDate.columns[len(getOnlyDate.columns) - 1])
        
        gini = giniData.melt(id_vars=['Country Name'], value_vars=[str(annee) for annee in range(firstDateColumnGini, lastDataColumnGini)], var_name='year', value_name='value')
        gini = gini.sort_values(['Country Name','year'])
        return gini


In [210]:
# le dataframe est illisible on va le réorganiser

,Country Name,year,value
0,ABW,1960,NaN
267,ABW,1961,NaN
534,ABW,1962,NaN
801,ABW,1963,NaN
1068,ABW,1964,NaN
...,...,...,...
15217,ZWE,2016,44.3
15484,ZWE,2017,NaN
15751,ZWE,2018,50.3
16018,ZWE,2019,NaN


In [185]:
giniByYear = gini.dropna(subset=['value']).reset_index().drop(columns=['index'])
giniByYear

,Country Name,year,value
0,AGO,1999,52.0
1,AGO,2007,42.7
2,AGO,2017,51.3
3,ALB,1995,27.0
4,ALB,2001,31.7
...,...,...,...
1840,ZMB,2009,55.6
1841,ZMB,2014,57.1
1842,ZWE,2010,43.2
1843,ZWE,2016,44.3


In [196]:
gini[gini['Country Name'] =='KHM' ]

,Country Name,year,value
123,KHM,1960,NaN
390,KHM,1961,NaN
657,KHM,1962,NaN
924,KHM,1963,NaN
1191,KHM,1964,NaN
...,...,...,...
15075,KHM,2016,NaN
15342,KHM,2017,NaN
15609,KHM,2018,NaN
15876,KHM,2019,NaN


In [192]:
iso3.head()

,name,alpha-2,iso
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM
